In [2]:


import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms
from tqdm import tqdm
import numpy as np

In [3]:
!pip install pytorch-metric-learning

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 1.7 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [37]:
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(512,512)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
!pwd

/home/ubuntu/study


In [6]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060', major=8, minor=6, total_memory=12044MB, multi_processor_count=28)

In [7]:
model_state_dict = torch.load('rn18_200epochs.pth',map_location='cuda:0')

In [8]:
model.load_state_dict(model_state_dict['rn18'])


<All keys matched successfully>

In [9]:
class_order = model_state_dict['class_order']

In [10]:
class_order

tensor([89,  0, 70, 84, 99, 65, 63, 60, 67, 61, 80, 87, 92,  9, 37, 35,  8, 93,
        21, 57, 53, 25, 43, 68, 88, 10, 30, 46, 39, 14, 22, 82, 83, 20, 96, 58,
         7,  5, 78, 38, 81, 71, 56, 32, 72, 31, 34, 45, 16, 33, 13, 29, 41, 49,
        86, 76, 64, 15, 94, 26, 55, 79, 52,  2, 91, 11, 17, 44, 74,  4, 24, 73,
        77, 98, 28, 95, 27, 47, 48, 42, 69, 40, 18, 36, 97,  3,  6, 85, 12,  1,
        66, 51, 62, 54, 50, 90, 75, 19, 23, 59], device='cuda:0')

In [11]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
])

In [12]:
ds_train = datasets.CIFAR100(root='./data', train=True, download=True, transform=preprocess)
ds_test  = datasets.CIFAR100(root='./data', train=False, download=True, transform=preprocess)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
def instances_from_classes(dataset, class_order):
    subset = []
    for i in range(len(dataset)):
        _, label = dataset[i]
        if label in class_order:
            subset.append(i)
    return subset


In [14]:
set_train = instances_from_classes(ds_train,class_order[:50])
set_test  = instances_from_classes(ds_test, class_order[:50])

In [15]:
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(set_train))
dl_test = torch.utils.data.DataLoader(ds_test,  batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(set_test))

In [16]:
x,y = next(iter(dl_train))

In [17]:
y.shape

torch.Size([64])

In [18]:
y

tensor([38,  5, 83, 32, 61, 67,  5, 58, 38, 30, 84,  5, 58, 35, 96, 99, 34, 83,
        53,  7, 87,  7, 21, 43, 21, 39, 61, 63, 10, 67, 30, 25, 92, 53, 87, 80,
         9, 61, 80, 61, 33, 22, 45, 70, 96, 34, 87, 68, 65, 72, 83, 92, 25, 43,
        31, 38, 57, 71, 81, 53,  9, 83, 30,  0])

In [19]:
from pytorch_metric_learning import losses
loss_func = losses.SubCenterArcFaceLoss(num_classes=100,embedding_size=512)

In [20]:
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 10.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 103.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [38]:
import wandb

In [39]:
wandb.init(
    # set the wandb project where this run will be logged
    project="clip+dmil",
)

loss_train,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_train,0.00419


In [42]:

opt = optim.Adam(model.parameters(),lr=0.001)

In [43]:
scheduler = optim.lr_scheduler.MultiStepLR(opt,milestones=[45,90],gamma=0.1)

In [44]:
device = torch.device('cuda:0')

In [45]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [46]:
emb = model(x.to(device))

In [47]:
y

tensor([38,  5, 83, 32, 61, 67,  5, 58, 38, 30, 84,  5, 58, 35, 96, 99, 34, 83,
        53,  7, 87,  7, 21, 43, 21, 39, 61, 63, 10, 67, 30, 25, 92, 53, 87, 80,
         9, 61, 80, 61, 33, 22, 45, 70, 96, 34, 87, 68, 65, 72, 83, 92, 25, 43,
        31, 38, 57, 71, 81, 53,  9, 83, 30,  0])

In [48]:
emb.shape

torch.Size([64, 512])

In [49]:
y.shape

torch.Size([64])

In [50]:
loss = loss_func(emb, y)

In [51]:
model.to(device)
loss_func.to(device)
loss_train = []
for epoch in range(100):
    model.train()
    loop = tqdm(dl_train)
    lloss = []
    for data, labels in loop:
        data = data.to(device)
        labels = labels.to(device)
        opt.zero_grad()
        embeddings = model(data)
        loss = loss_func(embeddings, labels)
        loss.backward()
        lloss.append(loss.item())
        opt.step()
    scheduler.step()
    loss_train.append(np.mean(lloss))
    wandb.log({'loss_train':loss_train[-1]})

100%|██████████| 391/391 [01:11<00:00,  5.46it/s]


In [52]:
torch

<module 'torch' from '/home/ubuntu/.local/lib/python3.10/site-packages/torch/__init__.py'>

In [53]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060', major=8, minor=6, total_memory=12044MB, multi_processor_count=28)

In [54]:
torch.save({'rn18':model.state_dict(),'class_order':class_order,'loss_train':loss_train},'rn18_pass_100epochs.pth')